# <font color = 'black'> <b>The Pain Killers — The Impact of Opioid Prescriptions on Overdose Deaths</b>
#### <font color = 'gray'> SIADS 593 - Milestone I - Spring/Summer, 2023
#### <font color = 'gray'> Douglas Bailley (dbailley), Ayan Banerjee (ayandsum), Richard Chalker (rchalker)

## <font color = 'steelblue'> <b>Introduction</b>
#### <font color = 'gray'> According to the Centers for Disease Control and Prevention (CDC), in 2020, 16,000 people died from an overdose of prescription opioids. That's 44 people per day who died from prescription painkillers, not street drugs.<br><sub>(Source: <https://www.cdc.gov/drugoverdose/deaths/prescription/maps.html>)</sub><br><br>This project aims to understand the rates of opioid prescriptions and overdose deaths in the population being studied. Specifically, we investigate legal opioid prescriptions and overdose deaths at the state and county level. This exploratory data analysis provides visualizations and statistics to illustrate observed trends.


## <font color = 'steelblue'> <b>Motivation</b>
#### <font color = 'gray'> For generations, heroin was widely considered the deadliest illegal narcotic available. But perceptions have changed in recent years. As the figure below shows, the rate of overdose deaths from prescription opioids has now surpassed the rate of overdose deaths from heroin.

![Altair motivation viz](assets/motivation_viz.png)

*Figure: Visualization illustrating the motivation behind this study.*

##### <font color='gray'><b>Altair visualization adapted from the National Center for Health Statistics data brief titled "NCHS Data Brief, Number 457, December 2022". <br><sub>(source: https://www.cdc.gov/nchs/data/databriefs/db457-tables.pdf#4n)</sub>

## <font color = 'steelblue'> <b>Configurations - <i>Imports, Initializations, & Standardization<i>

##### <font color = 'gray'>  Note to Coursera users: This notebook was validated on UMSI https://www.coursera.org/learn/siads622/<br><sub>required: pip install geopandas & pip install openpyxl<sub>

In [ ]:
# Import Pandas, NumPy, RegEx (re), and Zipfile libraries for data import and manipulation
from IPython.display import display
import numpy as np
import pandas as pd
import re
import statsmodels.formula.api as smf
import zipfile

# Import Altair and other required libraries for visualizations
import altair as alt
import geopandas as gpd
from io import StringIO
import json
import requests
from vega_datasets import data

# Import Sys to print library version information
import sys

In [ ]:
# Import custom functions from Python script "pk_functions" to keep notebook organized and readable
sys.path.append("./assets/")

from pk_functions import *

# keep warnings from cluttering the notebook
import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None

# state abbreviations (from pk_functions.py)
state_abbreviations = get_state_abbreviations()

<font color="black"> Recommend — Python version: 3.8.16 or higher, Pandas version: 1.4.4, Altair version: 4.2.0

In [ ]:
# Print version information. Recommend Python version: 3.8.16 or higher, Pandas version: 1.4.4, Altair version: 4.2.0
print(f"Python version: {sys.version[:6]}")
print(f"Pandas version: {pd.__version__}")
print(f"Altair version: {alt.__version__}")

In [ ]:
# Configure file paths
cdc_file_path = "data/Underlying_Causeof_Death_1999_2020_on_2023_5_12.txt"
cms_file_path = "data/Medicare_Part_D_Opioid_Prescribing_Rates_by_Geography_2020.zip"
cms_file_name = "Medicare_Part_D_Opioid_Prescribing_Rates_by_Geography_2020.csv"
# excel_file_path = "data/merged_df-test.xlsx"

## <font color = 'steelblue'> <b>Datasets Acquisition Process and Data Loading<b>

### <font color = 'steelblue'> <b>Dataset #1: </b><i>CDC Underlying Cause of Death, 1999-2020</i></font>
<font color = "gray">
URL is https://wonder.cdc.gov/ucd-icd10.html
Click "I Agree" towards bottom of page

Options on Request Form (menu selections)
1. Organize table layout
    Group Results By<br>
    select: "State"<br>
    select: "County"<br>
    select: "Year"<br><br>
2. Select location
    radio button: "States"<br>
    select: "All (The United States)"<br>
    radio button: 2013 Urbanization --> "All Categories"<br><br>
3. Select demographics
    radio button: "Five-Year Age Groups" --> "All Ages"<br>
    Gender: "All Genders"<br>
    Hispanic Origin: "All Origins"<br>
    Race: "All Races"<br><br>
4. Select year and month
    select: "All (All Dates)"<br><br>
5. Select weekday, autopsy and place of death
    Select: Weekday --> "All Weekdays"<br>
    Select: Autopsy --> "All Values"<br>
    Select: Place of Death --> "All Places<br><br>
6. Select cause of death:
    Radio button: "Drug/Alcohol Induced Causes"<br>
    Select: "Drug-Induced causes"<br><br>
7. Other options:
    Export Result: &#x2611;<br>
    Show Zero Values: &#x2611;<br>
    Show Supressed Values: &#x2611;<br>
    Precision: 2 decimal places<br>
    Data Access Timeout: 10 minutes

<font color="black"> Optional: The user can click on this Markdown cell and uncomment line below to see a .png with selections
<!-- ![CDC](assets/screencapture-wonder-cdc-gov-controller-datarequest-D76-2023-05-12-15_09_37.png) -->

In [ ]:
# use Pandas to read/load the CDC txt file on Opioid deaths
cdc_df = pd.read_csv(cdc_file_path, delimiter="\t", low_memory=False)

### <font color='steelblue'> <b>Dataset #2: </b><i>CMS Medicare Part D Opioid Prescribing Rates by Geography 2020</i> </font>
<font color="gray"> source: [https://data.cms.gov/](https://shorturl.at/fhwxY) </font>

In [ ]:
# use "zipfile" and Pandas to import the CMS data on Opioid Prescribing Rates
with zipfile.ZipFile(cms_file_path, "r") as zf:
    with zf.open(cms_file_name) as csv_file:
        cms_df = pd.read_csv(csv_file)

## <font color = 'steelblue'> <b>Data Description and Preparation<b>

### <font color = 'gray'> <b>CDC Data - <b><i>Underlying Cause of Death, 1999-2020</i>

<font color="black"> cdc_df before cleaning up

In [ ]:
# An overview of the cdc_df dataFrame
# cdc_df before cleaning up
print(f"cdc_df number of rows: {cdc_df.shape[0]}")
print(f"cdc_df number of columns: {cdc_df.shape[1]}")
print(f"cdc_df columns: {cdc_df.columns.tolist()}")

# Calculate the size of the Pandas DataFrame in MB (from pk_functions.py)
mem_MB = calculate_dataframe_memory_usage(cdc_df)
print(f"cdc_df 'Memory usage: {mem_MB:.2f} MB")

In [ ]:
# Dataset Notes start at row 69234  — prune them
cdc_df = cdc_df.iloc[:69234]

# Retain columns of interest
cdc_df = cdc_df[["State", "County", "County Code", "Population", "Year", "Deaths"]]

# use a subset of the data for the analysis
cdc_df.drop(cdc_df[cdc_df.Year < 2013].index, inplace=True)  # using 2013 - 2020 data
print(f"cdc_df number of rows: {cdc_df.shape[0]}")
print(f"cdc_df number of columns: {cdc_df.shape[1]}")
print(f"cdc_df columns: {cdc_df.columns.tolist()}")

cdc_original_df = cdc_df.copy()  # may be useful later

<font color="black">Our approach for treating "suppressed" and "missing" values is discussed below.<br>
Data imputation:  use the mean value (integer) between 1-9 to fill in the Suppressed data (value = 5) and Missing data (value = 0).

In [ ]:
# replace suppressed values with 5
cdc_df["Deaths"] = cdc_df["Deaths"].replace("Suppressed", 5)
# replace missing values with 0
cdc_df["Deaths"] = cdc_df["Deaths"].replace("Missing", 0)

# get the sum for cdc_df["Deaths"]
cdc_df["Deaths"] = cdc_df["Deaths"].astype(int)
# cdc_df["Deaths"].sum() # 574291

In [ ]:
# Set the data types for the columns of interest
dtype_dict = {
    "State": str,
    "County": str,
    "County Code": int,
    "Year": int,
    "Deaths": int,
}

cdc_df = cdc_df.astype(dtype_dict)

In [ ]:
# a check
cdc_df["Deaths"].sum()  # 574291

<font color="black"> An overview of the cleaned up cdc_df dataFrame

In [ ]:
# An overview of the cleaned up cdc_df dataFrame
display_dataframe_overview(cdc_df) # from pk_functions.py

### <font color = 'steelblue'> <b>CMS Data - <b><i>Medicare Part D Opioid Prescribing Rates by Geography</i>

<font color="black"> cms_df before cleaning up

In [ ]:
# cms_df before cleaning up
# An overview of the cms_df dataFrame
print(f"cms_df number of rows: {cms_df.shape[0]}")
print(f"cms_df number of columns: {cms_df.shape[1]}")

# cms_df columns is long — split it into two strings for readability without scrolling
print_column_names(cms_df) # from pk_functions.py

# Calculate the size of the Pandas DataFrame in MB (from pk_functions.py)
mem_MB = calculate_dataframe_memory_usage(cms_df)
print(f"cms_df 'Memory usage: {mem_MB:.2f} MB")


In [ ]:
# Retain year data for year 2020 and rows where 'Prscbr_Geo_Lvl' indicates 'ZIP'
# cms_df = cms_df[(cms_df["Year"] == 2020) & (cms_df["Prscrbr_Geo_Lvl"] == "ZIP")]
cms_df = cms_df[(cms_df["Prscrbr_Geo_Lvl"] == "ZIP")]
# cms_df

In [ ]:
# Create a copy of the DataFrame
cms_df = cms_df.copy()  #

# Apply the split_state_county_zip() function (an import from  pk_functions) to create new columns
cms_df["County"] = cms_df["Prscrbr_Geo_Desc"].apply(
    lambda x: split_state_county_zip(x, re, state_abbreviations)[0]
)
cms_df["Zip_Code"] = cms_df["Prscrbr_Geo_Desc"].apply(
    lambda x: split_state_county_zip(x, re, state_abbreviations)[1]
)

# drop rows where County is NaN
cms_df = cms_df.dropna(subset=["County"])

<font color="black"> An overview of the cleaned up cms_df dataFrame

In [ ]:
# An overview of the cleaned up cms_df DataFrame
display_dataframe_overview(cms_df) # from pk_functions.py

Question 1 - New

### <font color = 'steelblue'> <b>Question 1: </b> Is there a relationship between <i>"opioid related deaths"</i> and <i>"opioid prescription claims" at the state - level</i>?

In [ ]:
# create a version of cdc_df with a County-Year column. DataFrame:  cdc_c_y with only the columns of interest
cdc_c_y = cdc_df.copy()
cdc_c_y = cdc_c_y[["State", "County", "County Code", "Year", "Population", "Deaths"]]
cdc_c_y = cdc_c_y.rename(columns={"County Code": "County_Code"})
cdc_c_y.head(3)

# prepare for merge
cdc_c_y["County-Year"] = cdc_c_y["County"] + "-" + cdc_c_y["Year"].astype(str)
cdc_c_y.head(3)

In [ ]:
# create a version of cms_df with a County-Year column. DataFrame:  cms_c_y with only the columns of interest
cms_c_y = cms_df.copy()
cms_c_y = cms_c_y[["County", "Year", "Tot_Opioid_Clms"]]
cms_c_y.head(3)

# prepare for merge
cms_c_y["County-Year"] = cms_c_y["County"] + "-" + cms_c_y["Year"].astype(str)
cms_c_y.head(3)

In [ ]:
# Group by County and sum the total opioid claims in the county
groupby_object = cms_c_y.groupby(["County-Year"]).agg({"Tot_Opioid_Clms": "sum"})
opioid_clms_df = groupby_object.reset_index()

In [ ]:
# show the opioid_clms_df
opioid_clms_df.head(3)

In [ ]:
# a check
# show the total opioid claims in for County-Year which includes string "Weston County, WY"
print(opioid_clms_df[opioid_clms_df["County-Year"].str.contains("Weston County, WY")])  

# sum up: opioid_clms_df[opioid_clms_df["County-Year"].str.contains("Weston County, WY")
opioid_clms_df[opioid_clms_df["County-Year"].str.contains("Weston County, WY")] ["Tot_Opioid_Clms"].sum()

In [ ]:
# Merge cdc_df with cms_df on 'County-Year' column"
merged_df = cdc_c_y.merge(
    opioid_clms_df, left_on = "County-Year", right_on=  "County-Year", how = "left"
)

In [ ]:
# show merged_df
merged_df

In [ ]:
# change the name of final_df to new_final_df
new_final_df = merged_df[
    [
        "Year",
        "State",
        "County",
        "County_Code",
        "County-Year",
        "Population",
        "Tot_Opioid_Clms",
        "Deaths",
    ]
]


In [ ]:
# drop rows with NaN values in the "Tot_Opioid_Clms" column
new_final_df = new_final_df.dropna(subset=["Tot_Opioid_Clms"])
# new_final_df.head(3)

In [ ]:
# Set the data types for the columns of interest
dtype_dict = {
    "Year": int,
    "State": str,
    "County": str,
    "County_Code": str,
    "County-Year": str,
    "Population": int,
    "Tot_Opioid_Clms": int,
    "Deaths": int,
}

new_final_df = new_final_df.astype(dtype_dict)

In [ ]:
# Aggregate data at State level
grouped_state = new_final_df.groupby(["Year", "State"]).agg(
    {"Population": "sum", "Tot_Opioid_Clms": "sum", "Deaths": "sum"}
)

grouped_state = grouped_state.reset_index()

In [ ]:
# sort the 'grouped_state' DataFrame by the 'Deaths' column in descending order and reset the index
grouped_state = grouped_state.sort_values(by = ["Deaths"], ascending = False).reset_index(drop = True)

# create column for Opioid Deaths per 100,000 AND Opioid Claims per 100,000
grouped_state["Opioid_Deaths_per_100k"] = grouped_state["Deaths"] / grouped_state["Population"] * 100000
grouped_state["Opioid_Claims_per_100k"] = grouped_state["Tot_Opioid_Clms"] / grouped_state["Population"] * 100000

In [ ]:
# new column with (redundant) state sums
state_sum = grouped_state.groupby(["State"]).agg({"Tot_Opioid_Clms": "sum"})
grouped_state["state_sum"] = grouped_state["State"].map(state_sum["Tot_Opioid_Clms"]) 

In [ ]:
# a check to inspect, sort grouped state by year and state
# grouped_state[grouped_state["State"] == "California"]

grouped_state_sorted = grouped_state.sort_values(
    by=["State", "Year"], ascending=True
).reset_index(drop=True)

# a check - shows California
grouped_state_sorted[32:40]

### <font color = 'steelblue'><b>Visualizaton — question 1</b></br>

In [ ]:
# Create bar chart for opioid claims per state
bar_opioid_claims = (
    alt.Chart(grouped_state,
              title = alt.TitleParams(
                                text = "State-wise trend of Opioid Claims & Drug Overdose Deaths",
                                subtitle = ["Based on data from 2013 to 2020", " "],
                                anchor = 'start', fontSize = 20
                                , fontWeight = 'bold', subtitleFontSize = 16
                              )
              )
    .mark_bar()
    .encode(
        x = alt.X(
            "State:N",
            sort = alt.EncodingSortField(
                field = "Tot_Opioid_Clms", op = "sum", order = "descending"
            ),
        ),
        y = alt.Y("state_sum:Q", title = "Total Opioid Claims"),
    )
)

# Create a line graph for drug overdose deaths per state
line_total_deaths = (
    alt.Chart(grouped_state)
    .mark_bar(color="red", size=2 )
    .encode(
        x=alt.X(
            "State:N",
            sort = alt.EncodingSortField(
                field = "Tot_Opioid_Clms", op = "sum", order = "descending"
            ),
        ),
        y = alt.Y("Deaths:Q", title = "Total Deaths", axis = alt.Axis(titleColor = "red")),
    )
)

# Create a overlapping visualization for comparison
chart_claims_and_deaths = alt.layer(bar_opioid_claims, line_total_deaths).resolve_scale(y = "independent")
chart_claims_and_deaths

### <font color = 'steelblue'><b>Analysis — question 1</b></br>
#### <font color = 'gray'> The state-wise plotting of opioid claims and drug overdose deaths indicates that there's a strong positive correlation between total opioid claims and drug overdose death. The subsequent investigation of Pearson's correlation coefficient (0.88) corroborates this assessment. We also notice a statistically significant relationship between total opioid claims and number of drug overdose deaths.

### <font color = 'steelblue'> <b>Question 2: </b> Is there a relationship between <i>"opioid related deaths"</i> and <i>"opioid prescription claims"</i>?

### <font color = 'steelblue'> <b>Question 2: </b> EDA Preparation</i>?

### <font color = 'steelblue'><b>Visualizaton — Question 2</b></br>

In [ ]:
# Create bar chart for opioid claims per state per 100k population

bar_opioid_claims_100k = (
    alt.Chart(
        grouped_state,
        title=alt.TitleParams(
            text="State-wise comparison of Opioid Claims & Overdose Deaths per 100k Population",
            subtitle=["Based on data from 2013 to 2020", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_bar(width=10)
    .encode(
        x=alt.X(
            "State:N",
            sort=alt.EncodingSortField(
                field="Opioid_Claims_per_100k", op="sum", order="descending"
            ),
        ),
        y=alt.Y("Opioid_Claims_per_100k:Q"),
    )
)

# Create a line graph for opioid deaths per state per 100k population
line_opioid_deaths_100k = (
    alt.Chart(grouped_state)
    .mark_bar(color="red", size=2)
    .encode(
        x=alt.X(
            "State:N",
            sort=alt.EncodingSortField(
                field="Opioid_Claims_per_100k", op="sum", order="descending"
            ),
        ),
        y=alt.Y(
            "Opioid_Deaths_per_100k:Q",
            title="Overdose Deaths per 100k",
            axis=alt.Axis(titleColor="red"),
        ),
    )
)

# Create a overlapping chart for comparison
chart_claims_and_deaths_100k = alt.layer(
    bar_opioid_claims_100k, line_opioid_deaths_100k
).resolve_scale(y="independent")
chart_claims_and_deaths_100k.properties(width=800)

### <font color = 'steelblue'><b>Analysis — question 2</b></br>

In [ ]:
# This function is used to check strength and statistical significance of two features. (import from pk_functions.py)
corrStrengthSignificanceAnalysis(grouped_state, "Tot_Opioid_Clms", "Deaths")

#### <font color = 'gray'> While not impossible, it can be challenging to look at the numerical data alone and get a sense of how Opioid Prescription Claims and Opioid Overdose Deaths relate. To get a sense of the regional characteristics of the Opioid Crisis, a visualization was created. For each state, both Opioid Prescription Claims and Opioid Overdose Deaths were normalized by presenting the data on a per 100,000 basis. An Altair plot encodes "State" on the x-axis. On the left y-axis, "Opioid Claims per 100k" are encoded, and on the right y-axis, "Opioid Overdose Deaths per 100k".<br><br>The following observations can be made from the visualization:<br>Alabama has the most Opioid Prescription Claims, but not the most Opioid Overdose Deaths. Likewise, Arkansas and Oklahoma have similar patterns.<br>In contrast, Opioid Overdose Deaths far outpace Opioid Prescription Claims in West Virginia, Connecticut, and Alaska. This suggests that something other than just the prescription rate is behind the Opioid Crisis.<br>Alaska and Hawaii have similar Opioid Prescription Claims, but Alaska has a higher Opioid Overdose Death rate. Perhaps a future study will explore the connection between the number of daylight hours, climate, and drug abuse.<br><br>The Coefficient indicates a stong Positive correlation with statistical significance (p-value < 0.05>)


### <font color = 'steelblue'> <b>Question 3: </b> How opioid related deaths vary across counties in the USA</i>?</font>
<font color="gray">
The total number of death is color coded<br>
County names and corresponding death numbers are shown in tooltip (mouseover)<br>
The "white" areas indicate missing data for that county / geographic region

### <font color = 'gray'> <b>Prepare data for Choropleth map visualizations<b>
- Objective is to drill down at county level
- Reference: https://altair-viz.github.io/gallery/choropleth.html

In [ ]:
# Get counties
counties = alt.topo_feature(data.us_10m.url, "counties")

# Fetch the TopoJSON data
topojson_url = data.us_10m.url
response = requests.get(topojson_url)
topojson_data = json.loads(response.text)

# Convert TopoJSON to GeoJSON
geojson_data = gpd.read_file(StringIO(json.dumps(topojson_data)), driver="TopoJSON")

In [ ]:
# Merge Geojson data with final_df for start year 2013 on County code
# (represented as "id" in final_df and as "County_Code" in geojson_data)

geojson_data_plus_2013 = geojson_data.merge(
    new_final_df[new_final_df.Year == 2013],
    left_on="id",
    right_on="County_Code",
    how="left",
)

In [ ]:
# Merge Geojson data with final_df for final year 2020 on County code
# (represented as "id" in final_df and as "County_Code" in geojson_data)

geojson_data_plus_2020 = geojson_data.merge(
    new_final_df[new_final_df.Year == 2020],
    left_on="id",
    right_on="County_Code",
    how="left",
)

In [ ]:
# Add calculated columns for death per capita and total opioid claims per capita
# for year 2013

geojson_data_plus_2013["Deaths_Per_Capita"] = (
    geojson_data_plus_2013["Deaths"] / geojson_data_plus_2013["Population"]
)
geojson_data_plus_2013["Tot_Opioid_Clms_Per_Capita"] = (
    geojson_data_plus_2013["Tot_Opioid_Clms"] / geojson_data_plus_2013["Population"]
)

In [ ]:
# Add calculated columns for death per capita and total opioid claims per capita
# for year 2013

geojson_data_plus_2020["Deaths_Per_Capita"] = (
    geojson_data_plus_2020["Deaths"] / geojson_data_plus_2020["Population"]
)
geojson_data_plus_2020["Tot_Opioid_Clms_Per_Capita"] = (
    geojson_data_plus_2020["Tot_Opioid_Clms"] / geojson_data_plus_2020["Population"]
)

In [ ]:
# Review the data
geojson_data_plus_2020.head(3)

### <font color = 'steelblue'><b>Visualizaton — Question 3</b></br>

#### <font color = 'gray'> Here's a side-by-side comparison of absolute number of opioid related deaths in US counties in year 2013 vs 2020. We observe rise in the absolute numbers in 2020 in the counties which were already on higher side in 2013. 

In [ ]:
opi_death_2013 = (
    alt.Chart(
        geojson_data_plus_2013,
        title=alt.TitleParams(
            text="Number of opioid related deaths in US counties",
            subtitle=["in Year 2013", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_geoshape(stroke="lightgray")
    .encode(color="Deaths:Q", tooltip=["County:N", "Deaths:Q", "Tot_Opioid_Clms"])
    .project(type="albersUsa")
    .properties(width=500, height=300)
)

# Create the states plot
states = alt.topo_feature(data.us_10m.url, feature='states')

states_plot = alt.Chart(states).mark_geoshape(
    fill='none',
    stroke='gray'
).project('albersUsa')

opi_death_2013 = states_plot + opi_death_2013

# 2020
opi_death_2020 = (
    alt.Chart(
        geojson_data_plus_2020,
        title=alt.TitleParams(
            text="Number of opioid related deaths in US counties",
            subtitle=["in Year 2020", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_geoshape(stroke="lightgray")
    .encode(color="Deaths:Q", tooltip=["County:N", "Deaths:Q", "Tot_Opioid_Clms"])
    .project(type="albersUsa")
    .properties(width=500, height=300)
)

opi_death_2020 = states_plot + opi_death_2020

opi_death_2013 | opi_death_2020

### <font color = 'steelblue'><b>Analysis — Question 3</b></br>

 ### <font color = 'gray'>This analysis shows where there are concentrations of Opioid - related deaths across the United States in 2013 and 2020.<br>As can be seen, the concentrations are relatively consistent over time, but the darker colors reflect the growth in Opioid - related deaths.

### <font color = 'steelblue'> <b>Question 4: </b> How opioid related deaths and opioid claims vary across counties in the USA</i>?</font>
### <font color="gray">
- The total opioid claim numbers are color coded
- County names and corresponding death numbers are shown in tooltip (mouseover)
- The "white" areas indicate missing data for that county / geographic region

### <font color = 'steelblue'><b>Visualizaton — Question 4</b></br>

In [ ]:
opi_claims_2013 = (
    alt.Chart(
        geojson_data_plus_2013,
        title=alt.TitleParams(
            text="Total opioid claims in US counties",
            subtitle=["in Year 2013", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_geoshape(stroke="lightgray")
    .encode(color="Tot_Opioid_Clms:Q", tooltip=["County:N", "Deaths:Q", "Tot_Opioid_Clms:Q"])
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(
            geojson_data_plus_2013, "County_Code", ["Tot_Opioid_Clms"]
        ),
    )
    .project(type="albersUsa")
    .properties(width=500, height=300)
)
opi_claims_2013 = states_plot + opi_claims_2013

opi_claims_2020 = (
    alt.Chart(
        geojson_data_plus_2020,
        title=alt.TitleParams(
            text="Total opioid claims in US counties",
            subtitle=["in Year 2020", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_geoshape(stroke="lightgray")
    .encode(color="Tot_Opioid_Clms:Q", tooltip=["County:N", "Deaths:Q", "Tot_Opioid_Clms:Q"])
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(
            geojson_data_plus_2020, "County_Code", ["Tot_Opioid_Clms"]
        ),
    )
    .project(type="albersUsa")
    .properties(width=500, height=300)
)
opi_claims_2020 = states_plot + opi_claims_2020

opi_claims_2013 | opi_claims_2020

### <font color = 'steelblue'><b>Analysis — Question 4</b></br>
#### <font color = 'gray'> This comparative visualization of total number of opioid claims in 2013 vs 2020 in US counties reveals no significant change.

### <font color = 'steelblue'> <b>Question 5: </b> How opioid related deaths per capita varies across counties in the USA</i>?
- The death per capita is color coded
- County names and corresponding death numbers are shown in tooltip (mouseover)
- The "white" areas indicate missing data for that county / geographic region

In [ ]:
opi_death_per_capita_2013 = (
    alt.Chart(
        geojson_data_plus_2013,
        title=alt.TitleParams(
            text="Opioid deaths per capita in US counties",
            subtitle=["in Year 2013", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_geoshape(stroke="lightgray")
    .encode(
        color=alt.Color("Deaths_Per_Capita:Q", scale=alt.Scale(scheme="lightgreyred")),
        tooltip=["County:N", "Deaths:Q", "Tot_Opioid_Clms:Q"],
    )
    .project(type="albersUsa")
    .properties(width=500, height=300)
)
opi_death_per_capita_2013 = states_plot + opi_death_per_capita_2013

opi_death_per_capita_2020 = (
    alt.Chart(
        geojson_data_plus_2020,
        title=alt.TitleParams(
            text="Opioid deaths per capita in US counties",
            subtitle=["in Year 2020", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_geoshape(stroke="lightgray")
    .encode(
        color=alt.Color("Deaths_Per_Capita:Q", scale=alt.Scale(scheme="lightgreyred")),
        tooltip=["County:N", "Deaths:Q", "Tot_Opioid_Clms:Q"],
    )
    .project(type="albersUsa")
    .properties(width=500, height=300)
)
opi_death_per_capita_2020 = states_plot + opi_death_per_capita_2020

opi_death_per_capita_2013 | opi_death_per_capita_2020

### <font color = 'steelblue'><b>Analysis — Question 5</b></br>
#### <font color = 'gray'> Comparison of opioid deaths in US counties in 2013 vs 2020 indicates no noteworthy changes.

### <font color = 'steelblue'> <b>Question 6: </b> How opioid claims per capita and corresponding deaths vary across counties in the USA</i>?
- The opioid claims per capita is color coded
- County names and corresponding death numbers are shown in tooltip (mouseover)
- The "white" areas indicate missing data for that county / geographic region

In [ ]:
opi_claims_per_capita_2013 = (
    alt.Chart(
        geojson_data_plus_2013,
        title=alt.TitleParams(
            text="Opioid claims per capita in US counties",
            subtitle=["in Year 2013", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_geoshape(stroke="lightgray")
    .encode(
        color=alt.Color(
            "Tot_Opioid_Clms_Per_Capita:Q", scale=alt.Scale(scheme="lightgreyteal")
        ),
        tooltip=["County:N", "Deaths:Q", "Tot_Opioid_Clms:Q"],
    )
    .project(type="albersUsa")
    .properties(width=500, height=300)
)
opi_claims_per_capita_2013 = states_plot + opi_claims_per_capita_2013

opi_claims_per_capita_2020 = (
    alt.Chart(
        geojson_data_plus_2020,
        title=alt.TitleParams(
            text="Opioid claims per capita in US counties",
            subtitle=["in Year 2020", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_geoshape(stroke="lightgray")
    .encode(
        color=alt.Color(
            "Tot_Opioid_Clms_Per_Capita:Q", scale=alt.Scale(scheme="lightgreyteal")
        ),
        tooltip=["County:N", "Deaths:Q", "Tot_Opioid_Clms:Q"],
    )
    .project(type="albersUsa")
    .properties(width=500, height=300)
)
opi_claims_per_capita_2020 = states_plot + opi_claims_per_capita_2020

opi_claims_per_capita_2013 | opi_claims_per_capita_2020

#### <font color = 'gray'> The comparative analysis reveals no significant change in per capita opioid claims in US counties in 2013 vs 2020.

### <font color = 'steelblue'> <b>Question 7: </b>As the number of Prescriptions continues to rise, how does that affect Deaths per Prescriptions vs Opioid Prescriptions?

### <font color = 'steelblue'><b>Analysis — Question 7</b></br>
#### <font color = 'gray'>To answer this question, the CMS dataset was grouped by year and Opioid prescription claims were aggregated. The CDC dataset was treated similarly (aggregating for overdose deaths) and the two datasets were merged.<br>A visualization encodes Opioid Prescription Claims on the x-axis and Deaths per Prescription Claim on the y-axis. Altair’s  transform_regresssion() method was applied to the chart object to obtain a regression line.

#### <font color = 'gray'>Much of the current literature on Opiod - related deaths has to do with the decrease in legal Opioid prescriptions and the increase in Opioid deaths.  This data is validated by our countable data records and the CMS Opioid database as identified below.

#### <font color = 'gray'>One factor supporting this analysis is the regulations toughening Opioid prescriptions caused that decrease in prescriptions.  “In 2018, U.S. overdose death rates from natural and semi-synthetic opioids (i.e., prescription opioid painkillers) declined significantly compared with the prior year, as did those from the illicit opioid heroin. But death rates from synthetic opioids (e.g., fentanyl) again increased significantly—reaching yet another record high.” (SHADAC, P1)


In [ ]:
# prepare cms data for merging
cms__by_year = cms_df.groupby("Year").sum().reset_index()
cms_claims_by_year = cms__by_year[["Year", "Tot_Opioid_Clms"]]
# cms_claims_by_year

# prepare cdc data for merging
cdc_deaths_by_year = cdc_df.groupby("Year").sum().reset_index()
cdc_deaths_by_year.drop(columns=["County Code"], inplace=True)
# cdc_deaths_by_year

# merge cms and cdc data
claims_deaths_per_year = cms_claims_by_year.merge(cdc_deaths_by_year, on="Year")
# claims_deaths_per_year

# calculate deaths per claim
claims_deaths_per_year["Deaths_Per_Claim"] = (
    claims_deaths_per_year["Deaths"] / claims_deaths_per_year["Tot_Opioid_Clms"]
)
claims_deaths_per_year

### <font color = 'steelblue'><b>Visualizaton — Question 7</b></br>

In [ ]:
base = (
    alt.Chart(claims_deaths_per_year)
    .mark_circle(size=100)
    .encode(
        x=alt.X(
            "Tot_Opioid_Clms:Q",
            scale=alt.Scale(domain=[60_000_000, 84_000_000]),
            axis=alt.Axis(
                title="Opioid Prescription Claims", titleFontSize=14, labelFontSize=12
            ),
        ),
        y=alt.Y(
            "Deaths_Per_Claim:Q",
            scale=alt.Scale(zero=False),
            axis=alt.Axis(
                title="Deaths per Prescription Claim",
                titleFontSize=14,
                labelFontSize=12,
            ),
        ),
        tooltip=["Year:N", "Tot_Opioid_Clms:Q", "Deaths_Per_Claim:Q"],
    )
    .properties(
        width=750,
        height=500,
        title={
            "text": "Deaths per Prescriptions vs Opioid Prescriptions",
            "fontSize": 20,
        },
    )
)

regression = base.transform_regression(
    "Tot_Opioid_Clms", "Deaths_Per_Claim", method="linear"
).mark_line(color="red")

plot = base + regression
plot

### <font color = 'steelblue'><b>Analysis — Question 7</b></br>
#### <font color = 'gray'>To test the strength of the correlation between Opioid Prescription Claims and Deaths per Prescription the scipy.stats and statsmodels formula library were used. A strong negative correlation was observed as indicated by Pearson r:  -0.94 with a low p-value (statistically significant).  The Coefficient of Determination (“goodness of fit”) was R–squared 0.88 tells us that 88% of the variance in the dependent variable is predictable from the independent variable. 

In [ ]:
# show the regression correlation statistics
result = smf.ols(
    formula="Deaths_Per_Claim ~ Tot_Opioid_Clms", data=claims_deaths_per_year
).fit()
print_the_stats_2(claims_deaths_per_year, result, "Deaths_Per_Claim", "Tot_Opioid_Clms")

### <font color = 'steelblue'> <b>Question 8: </b>During 2013-2020 which counties saw the steepest rise in Overdose Deaths per Opioid Prescription?

### <font color = 'steelblue'><b>Analysis — Question 8</b></br>
#### <font color = 'gray'>The objective is  to identify the counties in the United States that saw the steepest rise in overdose deaths per opioid prescription between 2013 and 2020. The statsmodels formula library was used to create regression models.<br>The resulting table was filtered to only include rows with an R-squared value above 0.80. This was done to ensure that the results are only from counties where the linear regression model is a good fit for the data.<br>For reference the number of overdose deaths per 1000 opioid prescriptions in 2020 is shown in the table.


In [ ]:
# create a cdc_basic with only the columns of interest
cdc_basic = cdc_df.copy()
cdc_basic = cdc_basic[["County", "Year", "Deaths"]]
cdc_basic.head(3)

# prepare for merge
cdc_basic["County-Year"] = cdc_basic["County"] + "-" + cdc_basic["Year"].astype(str)
# cdc_basic.head(3)

In [ ]:
# create cms_basic which is a copy of cms_df_enriched with only the columns of interest
cms_basic = cms_df.copy()
cms_basic = cms_basic[["Year", "County", "Tot_Opioid_Clms"]]
cms_basic["County-Year"] = cms_basic["County"] + "-" + cms_basic["Year"].astype(str)

In [ ]:
# merge the two dataframes
basic_merged = cdc_basic.merge(
    cms_basic, on="County-Year", how="left", suffixes=("", "_y")
).loc[:, ["County", "Year", "Deaths", "Tot_Opioid_Clms"]]

In [ ]:
# create new column for deaths per 1000 opioid prescriptions
basic_merged["deaths_per_1000_prescriptions"] = (
    basic_merged["Deaths"] / basic_merged["Tot_Opioid_Clms"]
) * 1000

# Filter out records with zero variance in deaths_per_claim.
basic_merged2 = basic_merged[
    basic_merged.groupby("County")["deaths_per_1000_prescriptions"].transform("std") > 0
]

counties = basic_merged2["County"].unique()

# Get the stats
a_dict = {}
for each_county in counties:
    result = smf.ols(
        formula="deaths_per_1000_prescriptions ~ Year",
        data=basic_merged[basic_merged["County"] == each_county],
    ).fit()
    a_dict[each_county] = [
        result.params["Intercept"],
        result.params["Year"],
        result.rsquared,
    ]

# Create table
table = (
    pd.DataFrame.from_dict(
        a_dict, orient="index", columns=["Intercept", "coefficient", "R-squared"]
    )
    .reset_index()
    .rename(columns={"index": "County"})
)

# Filter:  only want rows with R-squared above 0.80
table = table[table["R-squared"] > 0.80]

# Sort the table descending by the absolute value of the coefficient
table["coefficient_abs"] = table["coefficient"].abs()
table = table.sort_values("coefficient_abs", ascending=False).drop(
    columns=["coefficient_abs"]
)

# Format the numbers for display (optional, if you just want to display the table)
table["Intercept"] = table["Intercept"].apply(lambda x: f"{x:.2f}")
table["coefficient"] = table["coefficient"].apply(lambda x: f"{x:.8f}")
table["R-squared"] = table["R-squared"].apply(lambda x: f"{x:.2f}")

In [ ]:
# create subset to prepare for merge
basic_merged_2020 = basic_merged[basic_merged["Year"] == 2020][
    ["County", "deaths_per_1000_prescriptions"]
]

# drop rows with NaN values
basic_merged_2020 = basic_merged_2020.dropna()

# rename column "deaths_per_1000_prescriptions" to "Year_2020_deaths_per_1000_prescriptions"
basic_merged_2020 = basic_merged_2020.rename(
    columns={"deaths_per_1000_prescriptions": "Year_2020_deaths_per_1000_prescriptions"}
)
# basic_merged_2020

# create a new DataFrame that is based on table and basic_merged_2020
basic_merged_2020 = pd.merge(table, basic_merged_2020, on="County", how="inner")

### <font color = 'steelblue'><b>Table — Question 8</b></br>

In [ ]:
# Display basic_merged_2020
print(
    "Table: \n\nSteepest Rise in Deaths per 1000 Prescriptions 2013-202\nThe Top 10 Counties\nby coefficient (slope) \nYear 2020 Deaths per 1000 Prescriptions is shown in the last column"
)
display(basic_merged_2020[:10].style.hide_index())

### <font color = 'steelblue'> <b>Question 9:</b> How should "suppressed" and "missing" values be treated?

#### <font color = 'gray'> Approach taken to address "Suppressed" death information</b>
Rational: For this analysis, The CDC has excluded all death counts under 9 deaths in a county.  The CDC's "Privacy policy: Statistics representing fewer than ten (one to nine) deaths or births are suppressed."*<br> (source:  https://wonmder.cdc.gov/wonder/help/ucd.html#Assurance%20of%20Confidentiality)

There are several solutions for understanding missing data.  Those include:
1) Missing Completely at Random (MCAR).  Given the CDC's guidance, we know that the Suppressed data is not random.
2) Missing at Random (conditionally) (MAR).  In this case, the missing data is related to the observation.  
3) Not Missing at Random (NMAR).  This is the case with the CDC dataset.  The decision to protect PII is what has driven that decision.

In our analysis, Approach 3 is the one that best describes our missing data.  However, given the knowledge we have about the data, we don't have too many options to impute the missing data.  For example, we have no comparison to county size since the values are small.  The population sizes for counties with Suppressed data ranges from the 50s to over 75000 residents.  We could look to a random weighted analysis based on population size, but in our review, we determined that population may not be the best weighting for the replacement.

As such, we will be populating the Suppressed Data values with 5.

Also, we have a portion of the data that is categorized as Missing.  We are treating that data as 0 deaths since the original dataset has explicit values from 1 up.  That covers 41 records, which is .2% percent of the records in the analysis.

Source:<br>
supressed is defined as 1-9 inclusive:  
*"Privacy policy: Statistics representing fewer than ten (one to nine) deaths or births are suppressed."*<br>
source:  https://wonmder.cdc.gov/wonder/help/ucd.html#Assurance%20of%20Confidentiality

In [ ]:
# EDA to determine approach for "Missing" values
death_values = list(cdc_original_df["Deaths"].unique())
death_values.sort()
print(f"Overview of Death Values:  {death_values[:5]} ... {death_values[-5:]}")

### <font color="gray"><b>EDA to determine approach for "Suppressed Deaths" values

In [ ]:
# Setting up a fresh dataframe for the analysis.
cdc_df_raw_supp = cdc_original_df.copy()

In [ ]:
# Evaluation of the missing values in the dataframe
suppressed_deaths = cdc_df_raw_supp.Deaths.str.count("Suppressed").sum()
missing_deaths = cdc_df_raw_supp.Deaths.str.count("Missing").sum()
suppressed_pop = cdc_df_raw_supp.Population.str.count("Suppressed").sum()
missing_pop = cdc_df_raw_supp.Population.str.count("Missing").sum()
total_rec = len(cdc_df_raw_supp)

In [ ]:
# Preparing the Suppressed Deaths dataframe
suppressed_deaths_df = cdc_df_raw_supp.loc[cdc_df_raw_supp["Deaths"] == "Suppressed"]
# print(suppressed_deaths_df)
suppressed_deaths_df = suppressed_deaths_df.drop(
    suppressed_deaths_df[(suppressed_deaths_df.Population == "Missing")].index
)
# suppressed_deaths_df['Population'].describe()

In [ ]:
# Preparing the Countable Deaths dataframe 
countable_deaths_df = cdc_df_raw_supp.loc[cdc_df_raw_supp["Deaths"] != "Suppressed"]
countable_deaths_df = countable_deaths_df.drop(
    countable_deaths_df[countable_deaths_df.Deaths == "Missing"].index
)
countable_deaths_df = countable_deaths_df.drop(
    countable_deaths_df[countable_deaths_df.Deaths == "nan"].index
)
countable_deaths_df.dropna(subset=["Deaths"])
countable_deaths_df.dropna(subset=["State"], inplace=True)
countable_deaths_df["Deaths"] = countable_deaths_df["Deaths"].astype(str).astype(int)
countable_deaths = countable_deaths_df["Deaths"].sum()
# countable_deaths_df['Deaths'].unique()

### <font color = 'steelblue'><b>Analysis — Question 9</b></br>

In [ ]:
print(
    "Summary (2013 - 2020): \n",
    "Total Records: ",
    total_rec,
    "\n",
    "Suppressed Deaths: ",
    suppressed_deaths,
    "Percent: %.3f" % (suppressed_deaths / total_rec),
    "\n",
    "Missing Deaths: ",
    missing_deaths,
    "Percent: %.3f" % (missing_deaths / total_rec),
    "\n",
    "Suppressed Population: ",
    suppressed_pop,
    "Percent: %.3f" % (suppressed_pop / total_rec),
    "\n",
    "Missing Population: ",
    missing_pop,
    "Percent: %.3f" % (missing_pop / total_rec),
    "\n",
)

print(
    "Options to address Suppressed Deaths: \n",
    "Total Countable (non-suppressed) Deaths: ",
    countable_deaths,
    "\n",
    "If Suppressed Deaths filled with 1 adds in: ",
    int(suppressed_deaths * 1),
    " \n",
    "Working Assumption: If Suppressed Deaths filled with 5 adds in: ",
    int(suppressed_deaths * 5),
    "for a total of",
    countable_deaths + int(suppressed_deaths * 5),
    "deaths. \n",
    "If Suppressed Deaths filled with 9 adds in: ",
    int(suppressed_deaths * 9),
    "\n",
)

### <font color = 'steelblue'> <b>Question 10: </b>During 2013-2020, what are the current trends in the ability to accurately track deaths in the CDC database?

In [ ]:
# Preparing the dataset
vis_countable_deaths_df = countable_deaths_df.copy()
vis_countable_deaths_df["Year"] = (
    vis_countable_deaths_df["Year"].astype(int).astype(str)
)
vis_countable_deaths_df = vis_countable_deaths_df.groupby(["Year"]).sum()
# vis_countable_deaths_df["Deaths"]



suppressed_deaths_df
suppressed_deaths_df["Year"] = suppressed_deaths_df["Year"].astype(int)
vis_suppressed_deaths_df = suppressed_deaths_df.groupby(["Year"]).count()
# vis_suppressed_deaths_df

### <font color = 'steelblue'><b>Visualization - Question 10

In [ ]:
bar_countable_deaths = (
    alt.Chart(
        vis_countable_deaths_df.reset_index(),
        title=alt.TitleParams(
            text="Countable Deaths Recorded",
            subtitle=["Based on data from 2013 to 2020", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_bar()
    .encode(
        x=alt.X("Year:O", title="Years"),
        y=alt.Y("Deaths:Q", title="Countable Deaths"),
    )
).properties(width=400)
# bar_countable_deaths

bar_suppressed_deaths = (
    alt.Chart(
        vis_suppressed_deaths_df.reset_index(),
        title=alt.TitleParams(
            text="Suppressed Death Record Counts",
            subtitle=["Based on data from 2013 to 2020", " "],
            anchor="start",
            fontSize=20,
            fontWeight="bold",
            subtitleFontSize=16,
        ),
    )
    .mark_bar()
    .encode(
        x=alt.X("Year:O", title="Years"),
        y=alt.Y("Deaths:Q", title="Suppressed Deaths"),
    )
).properties(width=400)
# bar_suppressed_deaths

# create a dummy Altair chart that is just a blank chart with the same size as the other two charts
blank_chart = ( 
    alt.Chart(vis_countable_deaths_df.reset_index())
    .mark_text()
    .properties(width=100)
)

plot = (bar_countable_deaths | blank_chart | bar_suppressed_deaths).configure_view(strokeWidth=0)

### <font color = 'steelblue'><b>Analysis — Question 10</b></br>
#### <font color = 'gray'>As seen below, deaths have increased from 2013 - 2020.  The visualization on the left refers to countable deaths (excluding our treatment for suppressed deaths).  As we have shown above, the relationship between deaths and prescriptions is dropping.  So how can we interpret this increase?

#### <font color = 'gray'>One possible explanation has been identified by the New York State Department of Health.  “From 2019 to 2020, there was another increase of 44 percent in overdose deaths involving any opioid (from 2,939 deaths to 4,233 deaths) among NYS residents, a 294 percent increase since 2010.” (NYSDOH, P15).  This supports the increase identified below.  But is this attributable to legally prescribed Opioids?  Our analysis in Question 7 indicates this is not the case.  In terms of driving the increase in deaths, there are a few contributors to the increase:

#### <font color = 'gray'>1) Per SHADAC, there has been a significant increase in deaths due to "synthetic Opioids" - those generally NOT medically prescribed (SHADAC, P3).  While their data is only current to 2018, the gap between 2013 and 2018 has dramatically increased.
#### <font color = 'gray'>2) States are getting better at recording deaths.  NY believes “it is possible that a portion of these observed increases in the past decade have likely been contributed by raised awareness of opioid overdoses, improvements in technology and resources for toxicology testing, and improved cause-of-death reporting”  (NYSDOH, P15)

#### <font color = 'gray'>Finally, what is apparent in our "Suppressed" record count, the increases in the deaths are probably not attributable to the declines in those record counts.  There are approximately 150 fewer "Suppressed" records in 2020 than in 2019.  If they all had 10 deaths and became reportable, that would only increase our "Countable Death" records by 1,500 deaths.  This is certainly not driving the increase as seen in the left visualization.


In [ ]:
# plot the viz
plot

NOTE: The Y-Axes are different scales.  The intent is to show the relative moves directionally over the analysis period.

#### <font color = 'gray'> These trends are being seen at the state level.  The NY “From 2019 to 2020, there was another increase of 44 percent in overdose deaths involving any opioid (from 2,939 deaths to 4,233 deaths) among NYS residents, a 294 percent increase since 2010.” (NYSDOH, P15).  

#### <font color = 'gray'> As well, the visualization on the right refers to a decrease in the "Suppressed" death number of records in the CDC database.  In essence, this means that counties are now recording "0" deaths or that the deaths have increased from 9, indicating that the death is no longer suppressed.

#### <font color = 'gray'>NY believes “it is possible that a portion of these observed increases in the past decade have likely been contributed by raised awareness of opioid overdoses, improvements in technology and resources for toxicology testing, and improved cause-of-death reporting”  (NYSDOH, P15)



### <font color = 'steelblue'>End</i>